In [4]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# --- 1. CONFIGURAR RUTAS ---
# Ajusta esto según dónde guardes el notebook
SHP_PATH = "data/geo_thailand/province_dd.shp"
CSV_PATH = "data/DengueThailand_2003-2024_Monthly.xlsx"

# --- 2. CARGAR MAPA (SHAPEFILE) ---
print("--- CARGANDO SHAPEFILE ---")
gdf = gpd.read_file(SHP_PATH)

# Ver qué columnas tiene para elegir la correcta
print("Columnas en el Mapa:", gdf.columns.tolist())

# INTENTO AUTOMÁTICO: Usar 'PROV_NAME' si existe, si no, buscar otra
col_mapa = 'PROV_NAME' if 'PROV_NAME' in gdf.columns else gdf.columns[1]
print(f"Usando columna del mapa: '{col_mapa}'")

# Crear llave normalizada (Mayúsculas y sin espacios)
gdf['KEY'] = gdf[col_mapa].astype(str).str.strip().str.upper()

# --- 3. CARGAR DATOS (CSV) ---
print("\n--- CARGANDO CSV ---")
df = pd.read_excel(CSV_PATH)
print("Columnas originales (primeras 5):", df.columns[:5].tolist())

# Transformar de Ancho a Largo (Melt)
df_melted = df.melt(id_vars=['Date'], var_name='province_raw', value_name='cases')

# Crear llave normalizada en el CSV
df_melted['KEY'] = df_melted['province_raw'].astype(str).str.strip().str.upper()

# --- 4. EL MOMENTO DE LA VERDAD: EL CRUCE ---
nombres_mapa = set(gdf['KEY'].unique())
nombres_csv = set(df_melted['KEY'].unique())

en_comun = nombres_mapa.intersection(nombres_csv)
solo_en_csv = nombres_csv - nombres_mapa # Nombres del CSV que NO encontró en el mapa
solo_en_mapa = nombres_mapa - nombres_csv # Nombres del Mapa que no tienen datos

print(f"\n✅ Coincidencias exactas: {len(en_comun)} provincias")
print(f"❌ Fallaron del CSV ({len(solo_en_csv)}): {sorted(list(solo_en_csv))}")
print(f"⚠️ Sobran en el Mapa ({len(solo_en_mapa)}): {sorted(list(solo_en_mapa))}")

--- CARGANDO SHAPEFILE ---
Columnas en el Mapa: ['OBJECTID', 'PROV_CODE', 'POLYTYPE', 'PROV_NAMT', 'PROV_NAME', 'YEAR_', 'MALE', 'FEMALE', 'TOTAL', 'HOUSE', 'SHAPE_AREA', 'SHAPE_LEN', 'geometry']
Usando columna del mapa: 'PROV_NAME'

--- CARGANDO CSV ---
Columnas originales (primeras 5): ['Date', 'Amnat Charoen', 'Ang Thong', 'PHRA NAKHON SI AYUTTHAYA', 'KRUNG THEP MAHA NAKHON (BANGKOK)']

✅ Coincidencias exactas: 76 provincias
❌ Fallaron del CSV (1): ['BUNGKAN']
⚠️ Sobran en el Mapa (0): []


In [11]:
POP_PATH = "data/population_thai.xlsx"

print("\n--- CARGANDO CSV ---")
df_pop = pd.read_excel(POP_PATH)
df_pop["PROV_NAME"]=df_pop["PROV_NAME"].str.lower()
gdf["PROV_NAME"]=gdf["PROV_NAME"].str.lower()
pd.merge(gdf,df_pop,on=["PROV_NAME"],how="left")


--- CARGANDO CSV ---


,OBJECTID,PROV_CODE,POLYTYPE,PROV_NAMT,PROV_NAME,YEAR_,MALE,FEMALE,TOTAL,HOUSE,...,SHAPE_LEN,geometry,KEY,Population,Area,Density,Capital,HS[7],ISO[8],FIPS
0,1,10,1.0000,กรุงเทพมหานคร,krung thep maha nakhon (bangkok),2001.0,2776975.0,2949228.0,5726203.0,1928921.0,...,264419.579302,"POLYGON ((100.33066 13.77279, 100.33074 13.773...",KRUNG THEP MAHA NAKHON (BANGKOK),NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,11,1.0000,สมุทรปราการ,samut prakan,2001.0,395819.0,415950.0,811769.0,298015.0,...,183154.515922,"POLYGON ((100.55225 13.69067, 100.55214 13.691...",SAMUT PRAKAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,12,1.0000,นนทบุรี,nonthaburi,2001.0,383969.0,417386.0,801355.0,326739.0,...,133785.673183,"POLYGON ((100.39878 13.80183, 100.39823 13.801...",NONTHABURI,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,13,1.0000,ปทุมธานี,pathum thani,2001.0,274192.0,290286.0,564478.0,236176.0,...,207204.451325,"POLYGON ((100.52674 14.12231, 100.52675 14.122...",PATHUM THANI,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,14,1.0000,พระนครศรีอยุธยา,phra nakhon si ayutthaya,2001.0,287064.0,301929.0,588993.0,159595.0,...,296496.611577,"POLYGON ((100.82321 14.27534, 100.82318 14.274...",PHRA NAKHON SI AYUTTHAYA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,72,92,1.4074,ตรัง,trang,2001.0,346177.0,350341.0,696518.0,164800.0,...,591411.904454,"MULTIPOLYGON (((99.70297 7.99066, 99.70378 7.9...",TRANG,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,73,93,1.0625,พัทลุง,phatthalung,2001.0,223194.0,229017.0,452211.0,113532.0,...,434507.129371,"MULTIPOLYGON (((100.1844 7.51417, 100.1862 7.5...",PHATTHALUNG,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,74,94,1.0000,ปัตตานี,pattani,2001.0,279048.0,287217.0,566265.0,115024.0,...,307412.574636,"POLYGON ((101.34502 6.89678, 101.3431 6.89893,...",PATTANI,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,75,95,1.0000,ยะลา,yala,2001.0,218033.0,217586.0,435619.0,101376.0,...,534984.346916,"POLYGON ((101.28036 6.68188, 101.2804 6.68186,...",YALA,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
import pandas as pd

# 1. CARGAR DATOS
# Ajusta las rutas a tus archivos reales
df_casos = pd.read_excel("data/DengueThailand_2003-2024_Monthly.xlsx")
df_pop = pd.read_excel("data/population_thai.xlsx")

# 2. PREPARAR LISTA DE NOMBRES DE CASOS
# Las columnas son provincias (saltando 'Date')
nombres_casos = [c for c in df_casos.columns if c != 'Date']
df_check = pd.DataFrame({'Caso_Original': nombres_casos})
# Normalizar (Mayúsculas y quitar espacios de los lados)
df_check['Caso_Norm'] = df_check['Caso_Original'].astype(str).str.strip().str.upper()

# 3. PREPARAR LISTA DE POBLACIÓN
# La columna suele tener un espacio al final "Name "
col_name_pop = 'Name ' if 'Name ' in df_pop.columns else 'Name'
df_pop['Pop_Norm'] = df_pop[col_name_pop].astype(str).str.strip().str.upper()

# 4. INTENTAR CRUZAR
merged = df_check.merge(df_pop, left_on='Caso_Norm', right_on='Pop_Norm', how='left')

# 5. VER ERRORES
print("--- PROVINCIAS QUE NO ENCONTRARON POBLACIÓN ---")
errores = merged[merged['Pop_Norm'].isna()]

if len(errores) > 0:
    display(errores[['Caso_Original', 'Caso_Norm']])
    print(f"\nTotal errores: {len(errores)}")
else:
    print("¡Todo perfecto! 100% Coincidencia.")

# 6. VER SUGERENCIAS (BÚSQUEDA EN POBLACIÓN)
print("\n--- SUGERENCIAS EN ARCHIVO DE POBLACIÓN ---")
# Buscamos nombres parecidos para los errores
for nombre_erroneo in errores['Caso_Norm']:
    # Buscamos algo que contenga parte del nombre
    parte = nombre_erroneo.split(' ')[0] 
    sugerencias = df_pop[df_pop['Pop_Norm'].str.contains(parte, na=False)]['Pop_Norm'].tolist()
    print(f"Para '{nombre_erroneo}' podría ser: {sugerencias}")

--- PROVINCIAS QUE NO ENCONTRARON POBLACIÓN ---


,Caso_Original,Caso_Norm
4,Bungkan,BUNGKAN
7,CHAINAT,CHAINAT
22,Lop Buri,LOP BURI
34,Nong Bua Lam Phu,NONG BUA LAM PHU
39,Phangnga,PHANGNGA
48,Prachin Buri,PRACHIN BURI
61,Si Sa Ket,SI SA KET



Total errores: 7

--- SUGERENCIAS EN ARCHIVO DE POBLACIÓN ---
Para 'BUNGKAN' podría ser: []
Para 'CHAINAT' podría ser: []
Para 'LOP BURI' podría ser: ['LOPBURI']
Para 'NONG BUA LAM PHU' podría ser: ['NONG BUA LAMPHU', 'NONG KHAI', 'RANONG']
Para 'PHANGNGA' podría ser: []
Para 'PRACHIN BURI' podría ser: ['PRACHINBURI']
Para 'SI SA KET' podría ser: ['KALASIN', 'NAKHON RATCHASIMA', 'NAKHON SI THAMMARAT', 'PHRA NAKHON SI AYUTTHAYA', 'SING BURI', 'SISAKET']


In [21]:
import pandas as pd
import geopandas as gpd

# --- 1. RUTAS (Ajusta si es necesario) ---
SHP_PATH = "data/geo_thailand/province_dd.shp"
POP_PATH = "data/population_thai.xlsx"

print("--- CARGANDO ---")

# A) CARGAR MAPA
gdf = gpd.read_file(SHP_PATH)
# Detectamos columna de nombre (PROV_NAME es la más común en Tailandia)
col_mapa = 'PROV_NAME' if 'PROV_NAME' in gdf.columns else gdf.columns[1]
gdf['KEY'] = gdf[col_mapa].astype(str).str.strip().str.upper()
print(f"✅ Mapa cargado: {len(gdf)} provincias.")

# B) CARGAR POBLACIÓN
df_pop = pd.read_excel(POP_PATH)
# Detectamos columna nombre (suele ser 'Name' o 'Name ')
col_pop = 'Name ' if 'Name ' in df_pop.columns else 'Name'
df_pop['KEY'] = df_pop[col_pop].astype(str).str.strip().str.upper()

# --- APLICAR CORRECCIONES PREVIAS (Para limpiar ruido conocido) ---
# Estas son las que ya detectamos antes, las aplicamos para ver qué queda real
correcciones_pop = {
    'BANGKOK': 'KRUNG THEP MAHA NAKHON (BANGKOK)',
    'CHAI NAT': 'CHAINAT',
    'BUENG KAN': 'NONG KHAI' # Si tu mapa es viejo (76 provs), Bueng Kan no existe.
}
df_pop['KEY'] = df_pop['KEY'].replace(correcciones_pop)
print(f"✅ Población cargada: {len(df_pop)} registros.")


# --- 2. EL CRUCE (AMBOS SENTIDOS) ---

set_mapa = set(gdf['KEY'])
set_pop = set(df_pop['KEY'])

comunes = set_mapa.intersection(set_pop)

print(f"\n🤝 COINCIDENCIAS: {len(comunes)} provincias cruzan perfecto.")

print("\n" + "="*60)
print("❌ SENTIDO 1: Provincias del MAPA que NO tienen Población")
print("   (Estas saldrán con Incidencia 0 o Error en el cálculo)")
print("="*60)
falla_en_mapa = sorted(list(set_mapa - set_pop))
if not falla_en_mapa:
    print("¡PERFECTO! Ninguna.")
else:
    for p in falla_en_mapa:
        print(f"   - {p}")

print("\n" + "="*60)
print("⚠️ SENTIDO 2: Provincias de la POBLACIÓN que NO están en el Mapa")
print("   (Datos que sobran y no se pintarán)")
print("="*60)
falla_en_pop = sorted(list(set_pop - set_mapa))
if not falla_en_pop:
    print("¡PERFECTO! Ninguna.")
else:
    for p in falla_en_pop:
        print(f"   - {p}")

--- CARGANDO ---
✅ Mapa cargado: 76 provincias.
✅ Población cargada: 77 registros.

🤝 COINCIDENCIAS: 76 provincias cruzan perfecto.

❌ SENTIDO 1: Provincias del MAPA que NO tienen Población
   (Estas saldrán con Incidencia 0 o Error en el cálculo)
¡PERFECTO! Ninguna.

⚠️ SENTIDO 2: Provincias de la POBLACIÓN que NO están en el Mapa
   (Datos que sobran y no se pintarán)
¡PERFECTO! Ninguna.
